# Análisis de regresión: variables dicotómicas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

## 9.1. Variables dicotómicas

El archivo *ubermty* contiene información de viajes en taxi o uber en la ciudad de Monterrey de junio de 2016 a agosto de 2017. Crearemos un modelo para estimar la duración de cada viaje en función de la distancia y el día de la semana.

In [ ]:
variables=['id', 'dropoff_longitude', 'dropoff_latitude', 
           'Dia', 'DuracionMinutos', 'DistanciaKm']

# Cargar los datos
df = pd.read_excel('https://github.com/adan-rs/AnalisisDatos/raw/main/datasets/ubermty.xlsx', usecols=variables)

In [ ]:
# Revisar las variables
df.info()

In [ ]:
# Obtener la estadística descriptiva
df.describe().T

In [ ]:
# Filtrado de valores atípicos
Q1 = df['DuracionMinutos'].quantile(0.25)
Q3 = df['DuracionMinutos'].quantile(0.75)
IQR = Q3-Q1
lim_inf = Q1-1.5*IQR
lim_sup = Q3+1.5*IQR
df = df[(df['DuracionMinutos']>lim_inf)&(df['DuracionMinutos']<lim_sup)]

In [ ]:
# Generar un gráfico
sns.scatterplot(x="dropoff_longitude",
                y="dropoff_latitude",
                data=df)

In [ ]:

import folium
map_center = [df['dropoff_latitude'].mean(), df['dropoff_longitude'].mean()]
mapa = folium.Map(location=map_center, zoom_start=10)
for index, row in df.iterrows():
    folium.CircleMarker(
        location=[row['dropoff_latitude'], row['dropoff_longitude']],
        radius=3, color='blue', fill=True, fill_color='blue', 
        fill_opacity=0.6 
    ).add_to(mapa)
mapa

In [ ]:
sns.boxplot(x='Dia', y='DuracionMinutos', data=df);

In [ ]:
df.head()

**Creación de variables dicotómicas**: La función `get_dummies` convierte una variable categórica (con k categorías) en k variables dicotómicas. Entre otros parámetros se puede establecer:
- prefix: agrega un prefijo a los nombres de las columnas.
- dtype: de manera predeterminada está en *booleano*, se puede cambiar a *int*
- drop_first: quita la columna con la primera categoría

Puedes usar esta función para crear un nuevo dataframe de las variables dummy o puedes aplicarlo directamente al dataframe:

In [ ]:
# Crear variables dicotómicas
df = pd.get_dummies(df, columns=['Dia'], prefix='dia',  
                    dtype=int, drop_first=True)
df.head(3)

In [ ]:
df.columns

In [ ]:
X = df[['DistanciaKm', 'dia_lunes', 'dia_martes', 'dia_miercoles', 
        'dia_jueves', 'dia_viernes', 'dia_sabado']]  
# Añadir una columna de unos para el intercepto
X = sm.add_constant(X)
y = df['DuracionMinutos']

In [ ]:
# Ajustar el modelo de regresión
model = sm.OLS(y, X).fit()

# Imprimir los resultados
print(model.summary())